In [1]:
import sys
import os
# Add the project root directory to Python path
sys.path.append('/Users/yusuf.bektas/Quant/drl-crypto')

In [2]:
import pandas as pd
df = pd.read_parquet("/Users/yusuf.bektas/Quant/drl-crypto/data/features/600s/ADAUSDT-600s-2024-06-09.parquet")
df.head()




,ts,price_last,buy_qty,sell_qty,best_bid,best_ask,log_ret_1_zscore_60,log_ret_5_zscore_300,log_ret_20_zscore_1200,log_volume_1_zscore_60,log_volume_5_zscore_300,log_volume_20_zscore_1200,ofi_1_zscore_60,ofi_5_zscore_300,ofi_20_zscore_1200
0,2024-06-09 11:10:00,0.4415,90380.6,129228.6,0.4415,0.4427,0.171787,0.378506,0.748720,-0.291292,-0.466866,-0.292758,-0.571453,0.555531,1.294796
1,2024-06-09 11:20:00,0.4419,105301.9,48792.1,0.4415,0.4419,0.467290,0.200261,0.972302,-0.766473,-0.676194,-0.405209,1.019764,0.132805,1.908748
2,2024-06-09 11:30:00,0.4418,55881.1,38100.3,0.4415,0.4418,-0.220243,-0.203527,0.990477,-1.400118,-1.068659,-0.523119,0.518410,-1.082912,2.160138
3,2024-06-09 11:40:00,0.4409,88368.3,121403.3,0.4409,0.4418,-1.325179,-0.314512,0.747384,-0.300113,-1.289748,-0.569406,-0.487371,-0.692684,1.882618
4,2024-06-09 11:50:00,0.4405,120016.5,115168.3,0.4405,0.4418,-0.609218,-0.086532,0.522533,-0.129656,-1.395926,-0.722999,0.028199,0.001517,1.274734


In [3]:
import pandas as pd
df2 = pd.read_parquet("/Users/yusuf.bektas/Quant/drl-crypto/data/features/600s/BTCUSDT-600s-2024-06-09.parquet")
df2.head()




,ts,price_last,buy_qty,sell_qty,best_bid,best_ask,log_ret_1_zscore_60,log_ret_5_zscore_300,log_ret_20_zscore_1200,log_volume_1_zscore_60,log_volume_5_zscore_300,log_volume_20_zscore_1200,ofi_1_zscore_60,ofi_5_zscore_300,ofi_20_zscore_1200
0,2024-06-09 11:10:00,69343.24,12.87772,14.99385,69343.24,69348.36,0.256725,0.109499,0.045567,-0.687486,-1.049667,-1.688654,0.052670,-1.688407,-2.882798
1,2024-06-09 11:20:00,69353.88,16.08182,19.89052,69343.24,69353.88,0.322142,0.174766,0.022577,-0.135774,-1.040784,-1.696766,-0.016876,-1.749858,-2.912060
2,2024-06-09 11:30:00,69368.96,15.48933,40.17119,69343.24,69368.96,0.489996,0.241636,0.039330,0.796398,-0.774392,-1.701003,-1.170980,-2.038032,-2.810793
3,2024-06-09 11:40:00,69368.01,14.65170,11.55857,69343.24,69368.01,-0.070989,0.223214,0.068345,-0.810503,-0.961186,-1.709710,0.779238,-1.257713,-2.577530
4,2024-06-09 11:50:00,69367.69,17.07303,23.18464,69343.24,69367.69,-0.058434,0.277577,0.033599,0.068630,-0.952082,-1.677583,-0.188825,-0.921043,-2.584554


In [5]:
import torch
from data_pipeline.rolling_dataset import RollingWindowDataset
from models.eiie_cnn import EIIE_CNN
from agents.eiie_agent import EIIEAgent
import pandas as pd
import numpy as np
import os

symbols = ["ETHUSDT", "BTCUSDT"]
ds = RollingWindowDataset("/Users/yusuf.bektas/Quant/drl-crypto/data/features/600s", symbols, window=10, sampling_freq="600s", start = "2025-05", end = "2025-06")


model = EIIE_CNN(m_assets=len(symbols),
                 in_channels=ds.C,
                 window=ds.n)

agent = EIIEAgent(model, commission=0.0002, lr=1e-5, batch_size=32)




log_equity_curve = []

counter = 0
log_equity = 0.0
for X, y in ds:
    r, w_t = agent.step(X, y)
    if counter % 500 == 0:
        print(f"Step {counter}, Reward: {r}, Log Equity: {log_equity}, w_t: {w_t}, y: {y}")
    log_equity += r

    log_equity_curve.append(log_equity)
    counter += 1
equity = np.exp(np.array(log_equity_curve))

print("Final equity factor:", equity)


Step 0, Reward: -0.001638502348214388, Log Equity: 0.0, w_t: tensor([1.0000e+00, 3.6559e-17]), y: tensor([0.9981, 0.9988])
Step 500, Reward: 0.0001802282058633864, Log Equity: -0.00440126591979606, w_t: tensor([1.0000e+00, 5.3251e-18]), y: tensor([1.0002, 1.0006])
Step 1000, Reward: 0.0051706633530557156, Log Equity: -0.013181441843926223, w_t: tensor([1.0000e+00, 2.7003e-42]), y: tensor([1.0052, 1.0041])
Step 1500, Reward: -0.004174110479652882, Log Equity: 0.27894089064807304, w_t: tensor([1.0000e+00, 4.1273e-31]), y: tensor([0.9958, 0.9977])
Step 2000, Reward: -0.0018670095596462488, Log Equity: 0.3168800463779462, w_t: tensor([1.0000e+00, 6.3516e-15]), y: tensor([0.9981, 0.9986])
Step 2500, Reward: 0.002257063053548336, Log Equity: 0.2579303152608645, w_t: tensor([1.0000e+00, 4.6646e-17]), y: tensor([1.0023, 1.0010])
Step 3000, Reward: -0.0011452435282990336, Log Equity: 0.28396424947572996, w_t: tensor([1., 0.]), y: tensor([0.9989, 0.9996])
Step 3500, Reward: 0.0019240929977968335